## Imports

In [10]:
# For plotting
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# To remove empty dates
import pandas as pd

# For reading properties
from jproperties import Properties

# Finnhub library
import finnhub

import datetime

## Load Properties

In [11]:
# Initialize from property file
configs = Properties()

with open('config/fh_analyst_trends.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
API_KEY = configs.get('FH_API_KEY').data
TEMPLATE  = configs.get('TEMPLATE').data

## Load Data

In [12]:
# Setup client
finnhub_client = finnhub.Client(api_key=API_KEY)

# Stock recommendations
res = finnhub_client.recommendation_trends(TICKER)

#Convert to Pandas Dataframe
df = pd.DataFrame(res)
df

,buy,hold,period,sell,strongBuy,strongSell,symbol
0,12,19,2023-08-01,2,16,0,NFLX
1,11,20,2023-07-01,3,15,0,NFLX
2,11,20,2023-06-01,3,15,0,NFLX
3,11,20,2023-05-01,3,15,0,NFLX


## Utility method to convert date to month year format

In [13]:
# Apply this method to convert dates to month year format
def add_month_year_col(x):
    date_obj = datetime.datetime.strptime(x, '%Y-%m-%d')
    return date_obj.strftime('%b') + ' ' + date_obj.strftime('%y')

## Add label column

In [14]:
df['label'] = df['period'].apply(add_month_year_col)
df

,buy,hold,period,sell,strongBuy,strongSell,symbol,label
0,12,19,2023-08-01,2,16,0,NFLX,Aug 23
1,11,20,2023-07-01,3,15,0,NFLX,Jul 23
2,11,20,2023-06-01,3,15,0,NFLX,Jun 23
3,11,20,2023-05-01,3,15,0,NFLX,May 23


## Sort by period

In [15]:
# Sort by dates - we plot chart in ascending order
df = df.sort_values('period')
df

,buy,hold,period,sell,strongBuy,strongSell,symbol,label
3,11,20,2023-05-01,3,15,0,NFLX,May 23
2,11,20,2023-06-01,3,15,0,NFLX,Jun 23
1,11,20,2023-07-01,3,15,0,NFLX,Jul 23
0,12,19,2023-08-01,2,16,0,NFLX,Aug 23


## Plot Analyst Trends

In [16]:
# Create bar charts
fig = go.Figure(data=[
    go.Bar(name='Strong Sell', x=df.label, y=df.strongSell, text=df.strongSell, marker_color='darkred'),
    go.Bar(name='Sell', x=df.label, y=df.sell, text=df.sell, marker_color='indianred'),
    go.Bar(name='Hold', x=df.label, y=df.hold, text=df.hold, marker_color='orange'),
    go.Bar(name='Buy', x=df.label, y=df.buy, text=df.buy, marker_color='#2ECC71'),
    go.Bar(name='Strong Buy', x=df.label, y=df.strongBuy, text=df.strongBuy, marker_color='forestgreen')
])
# Layout for the plot
layout = go.Layout(
    template=TEMPLATE,
    title=TICKER + ' - Recommendation Trends',
    # Stack bar
    barmode='stack',
    width=700, height=600,
    uniformtext=dict(
        minsize=10,
        mode='hide'
    ),
    xaxis=dict(
        tickfont_size=12
    ),
    yaxis=dict(
        title='# Analysts',
        titlefont_size=16,
        tickfont_size=12,
    ),
    # Horizontal ledger
    legend=dict(
        orientation="h"
    ),
    font=dict(
        size=12
    )
)
fig.update_layout(layout)

# Zero angle for the text inside bar and set the width between bars
fig.update_traces(textangle=0, width=0.5)

fig.show()

## Compute the Ratings
Reference:[Understanding Buy, Sell, and Hold Ratings of Stock Analysts](https://www.investopedia.com/financial-edge/0512/understanding-analyst-ratings.aspx)

In [17]:
df['rating'] = round((df.strongBuy * 1 + df.buy * 2 + df.hold * 3 + df.sell * 4 + df.strongSell * 5)
                    / (df.strongBuy + df.buy + df.hold + df.sell + df.strongSell),2)
df

,buy,hold,period,sell,strongBuy,strongSell,symbol,label,rating
3,11,20,2023-05-01,3,15,0,NFLX,May 23,2.22
2,11,20,2023-06-01,3,15,0,NFLX,Jun 23,2.22
1,11,20,2023-07-01,3,15,0,NFLX,Jul 23,2.22
0,12,19,2023-08-01,2,16,0,NFLX,Aug 23,2.14


## Utlity method to add an horizontal line

In [18]:
# Adds a horizontal dotted line with given annotation text in color
def add_hline(fig, pos, color, text):
    fig.add_hline(y=pos, line_color=color, line_dash='dot',
                  annotation_text=text, annotation_position='top right', annotation_font_size=8, annotation_font_color=color)

## Plot Ratings

In [20]:
fig = go.Figure()
# Add horizontal lines
add_hline(fig, 5, 'darkred', 'Strong Sell')
add_hline(fig, 4, 'indianred', 'Sell')
add_hline(fig, 3, 'orange', 'Hold')
add_hline(fig, 2, '#2ECC71', 'Buy')
add_hline(fig, 1, 'forestgreen', 'Strong Buy')

# Plot the scores
fig.add_trace(go.Scatter(x=df.label, y=df.rating, name='Rating', showlegend=True,
                         mode='lines+markers+text', text=df.rating))

layout=go.Layout(
    template=TEMPLATE,
    title=TICKER + ' - Recommendation Rating Score',
    xaxis=dict(
        tickfont_size=12
    ),
    width=600, height=500,
    yaxis=dict(
        title='Rating',
        titlefont_size=16,
        tickfont_size=12,
        # Set our own range for y axis
        range=[0,5]
    ),
    legend=dict(
        orientation="h"
    ),    
    font=dict(
        size=10
    )
)
fig.update_layout(layout)

fig.show()